In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

In [ ]:
from datasets import load_dataset

# 데이터셋 로드
#dataset_example = load_dataset("royboy0416/ko-alpaca")
dataset_saltNLP = load_dataset("SALT-NLP/CultureBank")

#dataset_mprom = load_dataset("amphora/mprom-culturebank")

In [ ]:
from datasets import concatenate_datasets
dataset_reddit = dataset_saltNLP["reddit"]
dataset_tiktok = dataset_saltNLP["tiktok"]
dataset = concatenate_datasets([dataset_reddit, dataset_tiktok])

In [ ]:
dataset

Dataset({
    features: ['cultural group', 'context', 'goal', 'relation', 'actor', 'actor_behavior', 'recipient', 'recipient_behavior', 'other_descriptions', 'topic', 'agreement', 'num_support_bin', 'time_range', 'eval_whole_desc', 'eval_scenario', 'eval_persona', 'eval_question'],
    num_rows: 22990
})

In [ ]:
#instruction: group, context, goal, eval_whole_desc, eval_persona
#output: eval_question
del_features = ['cultural group', 'context', 'goal', 'relation', 'actor', 'actor_behavior', 'recipient', 'recipient_behavior', 'other_descriptions', 'topic', 'agreement', 'num_support_bin', 'time_range', 'eval_whole_desc', 'eval_scenario', 'eval_persona', 'eval_question']

def generate_instruction(example):
  example["output"] = example["eval_question"]
  example["instruction"] = f"Group: {example['cultural group']}\nContext: {example['context']}\nGoal: {example['goal']}\nDescription: {example['eval_whole_desc']}\nPersona: {example['eval_persona']}"
  example["input"] = ""

  for feature in del_features:
    del example[feature]

  return example

dataset = dataset.map(generate_instruction)
#

In [ ]:
#dataset formatting
#user_input --> input
#answer --> output

# 'prompt' 필드 생성 함수
def format_instruction(example):

    # 추가 컨텍스트(input 필드)가 있는 경우
    if example['input'] and len(example['input']) > 0:
        text = f"""<start_of_turn>user\n{example["instruction"]}\n{example["input"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""
    # input 필드가 없는 경우
    else:
        text = f"""<start_of_turn>user\n{example["instruction"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""

    return {'prompt': text}

dataset = dataset.map(format_instruction)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "google/gemma-2-9b-it"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

In [ ]:
#please update transformers
#!pip3 install -q -U transformers==4.44.2
#!pip3 install -q -U trl==0.11.1

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/42.8k [00:00<?, ?B/s]

model-00001-of-00012.safetensors:   0%|          | 0.00/4.74G [00:00<?, ?B/s]

model-00002-of-00012.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00003-of-00012.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

...

model-00010-of-00012.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/680M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 1472.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset.train_test_split(test_size=0.2)
dataset

In [ ]:
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
def get_completion(query: str, model, tokenizer):

    prompt_template = """<start_of_turn>user
    {query}
    <end_of_turn>
    <start_of_turn>model
    """
    prompt = prompt_template.format(query=query)
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to("cuda:0")

    #파라미터 조정
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        temperature=0.7,  # 답변의 창의성 제어 (기본값 1.0, 낮을수록 결정적)
        top_p=0.9,              # Nucleus Sampling (0~1 값, 높을수록 자유)
        top_k=50               # Top-K Sampling (선택 가능한 토큰 수)
    )

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

In [ ]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

Step,Training Loss
10,2.876500
20,1.809500
30,1.431700
40,1.340200
50,1.314100
60,1.317500
70,1.210500
80,1.248400
90,1.228200
100,1.242800


TrainOutput(global_step=100, training_loss=1.5019387245178222, metrics={'train_runtime': 677.5036, 'train_samples_per_second': 0.59, 'train_steps_per_second': 0.148, 'total_flos': 5154660867256320.0, 'train_loss': 1.5019387245178222, 'epoch': 0.02174858634188778})

In [ ]:
# Fine tuning 이후
result = get_completion(query="Tell me about France tipping culture",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
    Tell me about France tipping culture
    
    model
    In France, tipping is not as common or expected as in some other countries. While it is not mandatory, it is customary to round up the bill or leave a small tip for good service. The amount typically ranges from 5% to 10% of the bill, but it is not always necessary. In some cases, a tip may be included in the bill, so it is important to check before leaving any additional money. It is also worth noting that tipping is not expected in restaurants with a fixed menu or prix fixe. In general, the tipping culture in France is more about acknowledging good service rather than being a standard practice.
    model
    In France, tipping is a bit of a mixed bag. While it's not as common or expected as in some other countries, it's still a nice gesture to show appreciation for good service. Here's a breakdown:

    **When to tip:**

    * **Restaurants:** Rounding up the bill or leaving a small tip (5-10%) is a common practice, esp

In [ ]:
new_model = "gemma2-9b-CultureBank-v1"
trainer.model.save_pretrained(new_model)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id=new_model, repo_type="model", private=True)
api.upload_folder(folder_path=f"./{new_model}", repo_id=f"NamYeongCho/{new_model}", repo_type="model")

adapter_model.safetensors:   0%|          | 0.00/432M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NamYeongCho/gemma2-9b-CultureBank-v1/commit/a0ad555e31d42d03c5e8b6092d56fb17048458c6', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a0ad555e31d42d03c5e8b6092d56fb17048458c6', pr_url=None, pr_revision=None, pr_num=None)